## Preprocess Data
 - Dropping less useful Data
 - Filling the missing values(Data Imputation)

In [20]:
import numpy as np 
import pandas as pd

In [21]:
df = pd.read_csv('./titanic.csv')

In [22]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [24]:
df_clean = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1)

In [25]:
df_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
le = LabelEncoder()
df_clean["Sex"] = le.fit_transform(df_clean["Sex"])

In [30]:
df_clean.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500
5,0,3,1,NaN,0,0,8.4583
6,0,1,1,54.0,0,0,51.8625
7,0,3,1,2.0,3,1,21.0750
8,1,3,0,27.0,0,2,11.1333
9,1,2,0,14.0,1,0,30.0708


In [34]:
df_clean = df_clean.fillna(df_clean["Age"].mean())

In [35]:
df_clean.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.000000,1,0,7.2500
1,1,1,0,38.000000,1,0,71.2833
2,1,3,0,26.000000,0,0,7.9250
3,1,1,0,35.000000,1,0,53.1000
4,0,3,1,35.000000,0,0,8.0500
5,0,3,1,29.699118,0,0,8.4583
6,0,1,1,54.000000,0,0,51.8625
7,0,3,1,2.000000,3,1,21.0750
8,1,3,0,27.000000,0,2,11.1333
9,1,2,0,14.000000,1,0,30.0708


In [38]:
X = df_clean.drop("Survived", axis=1)
Y = df_clean["Survived"]

In [41]:
print(X.shape)
print(Y.shape)

(891, 6)
(891,)


## Entropy and Information Gain

In [67]:
def entropy(col):
    terms = np.unique(col, return_counts=True)
    n = float(col.shape[0])
    
    ent = 0.0
    for t in terms[1]:
        p = t/n 
        ent += (-1*p*np.log2(p))

    return ent

In [68]:
col = np.array([1,0,0,1,0,1,1])
entropy(col)

0.9852281360342515

In [69]:
def divide_data(x_data, fkey, fval):
    x_left = pd.DataFrame([], columns=x_data.columns)
    x_right = pd.DataFrame([], columns=x_data.columns)

    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]

        if val > fval:
            x_right = x_right.append(x_data.loc[ix])
        else:
            x_left = x_left.append(x_data.loc[ix])

    return x_left, x_right

In [70]:
## fkey -- the feature
## fval -- the threshold value of that feature (Mean, average)
def information_gain(x_data, fkey, fval):
    left, right = divide_data(x_data, fkey, fval)

    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]

    if left.shape[0] == 0 or right.shape[0] == 0:
        return -10000
    
    i_gain = entropy(x_data.Survived) - (l*entropy(left.Survived) + r*entropy(right.Survived))
    return i_gain

In [71]:
## Testing the information gain function
for fx in X.columns:
    print(fx)
    print(information_gain(df_clean, fx, df_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.001158644038169343
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [85]:
class DecisionTree:

    ## Constructor
    def __init__(self, depth = 0, max_depth = 5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        ## target is the value for a particular node (preferrably Leaf Node)

    def train(self, X_train):
        features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
        info_gains = []

        for ix in features:
            i_gain = information_gain(X_train, ix, X_train[ix].mean())
            info_gains.append(i_gain)

        self.fkey = features[np.argmax(info_gains)]
        self.fval = X_train[self.fkey].mean()

        print("Making Tree with feature", self.fkey)

        ## Split the DATA
        left, right = divide_data(X_train, self.fkey, self.fval)
        left = left.reset_index(drop=True)
        right = right.reset_index(drop=True)

        ## Truly a leaf node, to stop there -->
        if left.shape[0] == 0 or right.shape[0] == 0:
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        
        ## Stop early when depth >= max_depth -->
        if self.depth >= self.max_depth:
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return

        ## Recursive Case
        self.left = DecisionTree(depth = self.depth + 1, max_depth = self.max_depth)
        self.left.train(left)

        self.right = DecisionTree(depth = self.depth + 1, max_depth = self.max_depth)
        self.right.train(right)

        ## Set target value at every node
        if X_train.Survived.mean() >= 0.5:
            self.target = "Survived"
        else:
            self.target = "Dead"
        return

    def predict(self, test):
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [79]:
d = DecisionTree()
d.train(df_clean)

Making Tree with feature Sex
Making Tree with feature Pclass
Making Tree with feature Pclass
Making Tree with feature Parch
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Parch
Making Tree with feature Age
Making Tree with feature Fare
Making Tree with feature Parch
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Fare
Making Tree with feature SibSp
Making Tree with feature Fare
Making Tree with feature Fare
Making Tree with feature Age
Making Tree with feature SibSp
Making Tree with feature Parch
Making Tree with feature Age
Making Tree with feature SibSp
Making Tree with feature Fare
Making Tree with feature Parch
Making Tree with feature Parch
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Fare
Making Tree w

KeyboardInterrupt: 

## Making Predictions

In [80]:
split = int(0.7*df_clean.shape[0])
train_data = df_clean[:split]
test_data = df_clean[split:]
test_data = test_data.reset_index(drop=True)

In [81]:
train_data.shape

(623, 7)

In [87]:
dt = DecisionTree()

In [83]:
dt.train(train_data)

Making Tree with feature Sex
Making Tree with feature Pclass
Making Tree with feature Age
Making Tree with feature SibSp
Making Tree with feature Pclass
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature SibSp
Making Tree with feature Parch
Making Tree with feature Pclass
Making Tree with feature SibSp
Making Tree with feature Fare
Making Tree with feature Parch
Making Tree with feature Age
Making Tree with feature Pclass
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Parch
Making Tree with feature SibSp
Making Tree with feature Fare
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Fare
Making Tree with feature Age
Making Tree with feature Age
Making Tree with feature Fare
Making Tree with feature Age
Making Tree with feature Parch
Making Tree with feature Fare
Making Tree with feature Fare
Making Tree with feature Fare
Making Tree with feature Age
Making Tree with feature Fare
Making 

In [89]:
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

KeyError: None